# Odoo queries wit ERPpeek and plain xmlrpc

## Using plain XMLRPC

In [1]:
from sys import path
from pprint import pprint
from xmlrpc.client import ServerProxy

path.append('/Users/loicjounot/Code/internal-tools')
pprint(path)

['',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python35.zip',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5/plat-darwin',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5/lib-dynload',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5/site-packages/setuptools-19.6.2-py3.5.egg',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5/site-packages',
 '/Users/loicjounot/anaconda/envs/internal-tools/lib/python3.5/site-packages/IPython/extensions',
 '/Users/loicjounot/.ipython',
 '/Users/loicjounot/Code/internal-tools']


### Authenticate

In [118]:
# Instantiate a connector (but just to read in authentication information from the YAML file)
odoo = OdooConnector()
client = odoo.db

In [92]:
# create a ServerProxy object
base_url = odoo.url + '/xmlrpc/2/'
models = ServerProxy(base_url + 'object')
models

<ServerProxy for valkfleet.odoo.com/xmlrpc/2/object>

In [93]:
# Use the ServerProxy object to authenticate
common = ServerProxy(base_url + 'common')
user_id = common.authenticate(
    odoo.database,
    odoo.username,
    odoo.password,
    {}
)
common

<ServerProxy for valkfleet.odoo.com/xmlrpc/2/common>

### Extract a table using search then read

In [97]:
# Search a table: i.e. grab a list of ids
claims_ids = models.execute_kw(
    odoo.database,
    user_id,
    odoo.password,
    'crm.claim',
    'search',
    [()]
)
claims_ids[:10]

[1290, 1205, 1167, 1162, 918, 423, 222, 187, 186, 169]

In [88]:
# Read a table: i.e. load the records using the list of ids
claims_records = models.execute_kw(
    odoo.database,
    user_id,
    odoo.password,
    'crm.claim',
    'read',
    [claims_ids]
)
claims_records[0]

{'__last_update': '2016-02-23 00:43:16',
 'action_next': False,
 'active': True,
 'categ_id': [46, 'Ops. Man. - Other'],
 'cause': 'we believe this is shahzad farooq making prank calls. disregarded',
 'company_id': [3, 'Valk Fleet UK LTD'],
 'create_date': '2016-02-18 19:40:59',
 'create_uid': [121, 'Pickering, Aaron'],
 'date': '2016-02-18 19:39:50',
 'date_action_next': False,
 'date_closed': '2016-02-23 00:43:17',
 'date_deadline': False,
 'description': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs. Thi',
 'display_name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'email_cc': False,
 'email_from': False,
 'id': 1290,
 'message_follower_ids': [1994, 215, 219],
 'message_ids': [163055, 162240, 161684, 161683],
 'message_is_follower': False,
 'message_last_post': False,
 'message_summary': ' ',
 'message_unread': False,
 'name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'partner_id

### Extract a table using search_read

In [94]:
claims_records = models.execute_kw(
    odoo.database,
    user_id,
    odoo.password,
    'crm.claim',
    'search_read',
    [()]
)
claims_records[0]

{'__last_update': '2016-02-23 00:43:16',
 'action_next': False,
 'active': True,
 'categ_id': [46, 'Ops. Man. - Other'],
 'cause': 'we believe this is shahzad farooq making prank calls. disregarded',
 'company_id': [3, 'Valk Fleet UK LTD'],
 'create_date': '2016-02-18 19:40:59',
 'create_uid': [121, 'Pickering, Aaron'],
 'date': '2016-02-18 19:39:50',
 'date_action_next': False,
 'date_closed': '2016-02-23 00:43:17',
 'date_deadline': False,
 'description': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs. Thi',
 'display_name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'email_cc': False,
 'email_from': False,
 'id': 1290,
 'message_follower_ids': [1994, 215, 219],
 'message_ids': [163055, 162240, 161684, 161683],
 'message_is_follower': False,
 'message_last_post': False,
 'message_summary': ' ',
 'message_unread': False,
 'name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'partner_id

## Using the ERPpeek wrapper provided by odoo
Read the docs! http://erppeek.readthedocs.org/en/latest/api.html#client-and-services

In [187]:
# Instantiate a connector to get the Odoo Client object
odoo = OdooConnector()
client = odoo.db

### The Client Object

In [151]:
# Find out what methods we can call
client_methods = [attribute for attribute in dir(odoo.db)   
                            if attribute[0] is not '_' 
                            and callable(getattr(client, attribute))]
client_methods

['access',
 'count',
 'create_database',
 'exec_workflow',
 'execute',
 'execute_kw',
 'field',
 'fields',
 'from_config',
 'install',
 'keys',
 'login',
 'model',
 'models',
 'modules',
 'read',
 'render_report',
 'report',
 'report_get',
 'reset',
 'search',
 'uninstall',
 'upgrade',
 'wizard']

In [125]:
# Find out about the client object properties 
attributes = [attribute for attribute in dir(odoo.db) 
                        if attribute[0] is not '_' 
                        and not callable(getattr(client, attribute))]

for attribute in attributes:
    key_value = '{key} = {value}'.format(key=attribute, value=getattr(client, attribute))
    print(key_value)

common = <Service 'https://valkfleet.odoo.com/xmlrpc/common'>
connect = None
context = None
db = <Service 'https://valkfleet.odoo.com/xmlrpc/db'>
major_version = 8.
server_version = 8.saas~6
user = odoo@valkfleet.com


In [171]:
client.modules()['installed'][:10]

['crm',
 'mail',
 'account_voucher',
 'barcodes',
 'account_accountant',
 'survey',
 'sale',
 'stock',
 'stock_account',
 'purchase']

### The Model object

In [136]:
claims_model = client.model('crm.claim')
claims_model

<Model 'crm.claim'>

In [138]:
claims_model.keys()

['action_next',
 'active',
 'categ_id',
 'cause',
 'company_id',
 'create_date',
 'create_uid',
 'date',
 'date_action_next',
 'date_closed',
 'date_deadline',
 'description',
 'email_cc',
 'email_from',
 'id',
 'message_follower_ids',
 'message_ids',
 'message_is_follower',
 'message_last_post',
 'message_summary',
 'message_unread',
 'name',
 'partner_id',
 'partner_phone',
 'priority',
 'ref',
 'resolution',
 'stage_id',
 'team_id',
 'type_action',
 'user_fault',
 'user_id',
 'website_message_ids',
 'write_date',
 'write_uid',
 'x_close_uid',
 'x_parent_ticket',
 'x_team']

In [142]:
claims_model.fields()

{'__last_update': {'change_default': False,
  'company_dependent': False,
  'depends': ['create_date', 'write_date'],
  'manual': False,
  'readonly': True,
  'required': False,
  'searchable': False,
  'sortable': False,
  'store': False,
  'string': 'Last Modified on',
  'type': 'datetime'},
 'action_next': {'change_default': False,
  'company_dependent': False,
  'depends': [],
  'help': '',
  'manual': False,
  'readonly': False,
  'required': False,
  'searchable': True,
  'sortable': True,
  'store': True,
  'string': 'Next Action',
  'translate': False,
  'type': 'char'},
 'active': {'change_default': False,
  'company_dependent': False,
  'depends': [],
  'help': '',
  'manual': False,
  'readonly': False,
  'required': False,
  'searchable': True,
  'sortable': True,
  'store': True,
  'string': 'Active',
  'type': 'boolean'},
 'categ_id': {'change_default': False,
  'company_dependent': False,
  'context': {},
  'depends': [],
  'domain': [],
  'help': '',
  'manual': False,


In [153]:
# Find out about the Model object properties 
model_methods = [attribute for attribute in dir(claims_model) 
                        if attribute[0] is not '_' 
                        and callable(getattr(claims_model, attribute))]
model_methods

['access',
 'browse',
 'common',
 'connect',
 'context',
 'count',
 'create',
 'create_database',
 'db',
 'exec_workflow',
 'execute',
 'execute_kw',
 'field',
 'fields',
 'from_config',
 'get',
 'install',
 'keys',
 'login',
 'major_version',
 'model',
 'models',
 'modules',
 'read',
 'render_report',
 'report',
 'report_get',
 'reset',
 'search',
 'server_version',
 'uninstall',
 'upgrade',
 'user',
 'wizard']

In [159]:
# There is a lot in common with the Client object
set(model_methods) & set(client_methods)

{'access',
 'count',
 'create_database',
 'exec_workflow',
 'execute',
 'execute_kw',
 'field',
 'fields',
 'from_config',
 'install',
 'keys',
 'login',
 'model',
 'models',
 'modules',
 'read',
 'render_report',
 'report',
 'report_get',
 'reset',
 'search',
 'uninstall',
 'upgrade',
 'wizard'}

In [160]:
# And there are a few that are specific to the Model
set(model_methods) - set(client_methods)

{'browse',
 'common',
 'connect',
 'context',
 'create',
 'db',
 'get',
 'major_version',
 'server_version',
 'user'}

### Using the model object to read an Odoo table

In [163]:
categ_46 = claims_model.search([['categ_id', '=', 46]])
categ_46[:5]

[1290, 1418, 1407, 1406, 1401]

In [170]:
records = claims_model.read([])
records[0]

{'__last_update': '2016-02-23 00:43:16',
 'action_next': False,
 'active': True,
 'categ_id': [46, 'Ops. Man. - Other'],
 'cause': 'we believe this is shahzad farooq making prank calls. disregarded',
 'company_id': [3, 'Valk Fleet UK LTD'],
 'create_date': '2016-02-18 19:40:59',
 'create_uid': [121, 'Pickering, Aaron'],
 'date': '2016-02-18 19:39:50',
 'date_action_next': False,
 'date_closed': '2016-02-23 00:43:17',
 'date_deadline': False,
 'description': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs. Thi',
 'display_name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'email_cc': False,
 'email_from': False,
 'id': 1290,
 'message_follower_ids': [1994, 215, 219],
 'message_ids': [163055, 162240, 161684, 161683],
 'message_is_follower': False,
 'message_last_post': False,
 'message_summary': ' ',
 'message_unread': False,
 'name': 'Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 'partner_id

### Returning a RecordList object

In [173]:
records_list = claims_model.browse([])
objects

<RecordList 'crm.claim,length=1351'>

In [179]:
# The Record object is like a dictionary, just that it maps attributes to the keys.
# This is not particularly useful for us since we tend to load everything into a 
# Pandas dataframe straight away
dir(records_list[0])

['_context',
 '_external_id',
 '_fields',
 '_keys',
 '_model',
 '_model_name',
 '_name',
 '_send',
 'action_next',
 'active',
 'categ_id',
 'cause',
 'company_id',
 'copy',
 'create_date',
 'create_uid',
 'date',
 'date_action_next',
 'date_closed',
 'date_deadline',
 'description',
 'email_cc',
 'email_from',
 'id',
 'message_follower_ids',
 'message_ids',
 'message_is_follower',
 'message_last_post',
 'message_summary',
 'message_unread',
 'name',
 'partner_id',
 'partner_phone',
 'priority',
 'read',
 'ref',
 'refresh',
 'resolution',
 'stage_id',
 'team_id',
 'type_action',
 'unlink',
 'user_fault',
 'user_id',
 'website_message_ids',
 'write',
 'write_date',
 'write_uid',
 'x_close_uid',
 'x_parent_ticket',
 'x_team']

In [178]:
records_list[0]._keys

['action_next',
 'active',
 'categ_id',
 'cause',
 'company_id',
 'create_date',
 'create_uid',
 'date',
 'date_action_next',
 'date_closed',
 'date_deadline',
 'description',
 'email_cc',
 'email_from',
 'id',
 'message_follower_ids',
 'message_ids',
 'message_is_follower',
 'message_last_post',
 'message_summary',
 'message_unread',
 'name',
 'partner_id',
 'partner_phone',
 'priority',
 'ref',
 'resolution',
 'stage_id',
 'team_id',
 'type_action',
 'user_fault',
 'user_id',
 'website_message_ids',
 'write_date',
 'write_uid',
 'x_close_uid',
 'x_parent_ticket',
 'x_team']

In [188]:
# Nice stuff: get all rows from a given field
records_list.name

['Wok and Go claims that Omid Qasmi came to the restaurant smelling of drugs.',
 "Driver didn't accept an order",
 'Driver did not swipe delivered',
 'Driver Not accepting orders',
 'Drivers are not turning up on time',
 'Rude to drivers',
 'Queens Chicken and Pizza wants to speak to account menager',
 'Not getting assigned orders',
 'Some orders not showing as completed on Syed´s app',
 "pizza's order upside down",
 'driver did not show up-called him, problem with app-suddenly it worked after my call',
 'Invoice Query',
 'Restaurant not happy with our service',
 'Driver swpied order as delivered.',
 'Driver creation',
 'Driver creation',
 'Drivers not starting shift on time. ',
 'bilal hedoun and joseph jacob late for shift and not answering phones',
 'Restaurant trying to contact Andrew sales rep for two weeks',
 'Driver has worked an extra hour',
 'Driver has worked extra hours',
 'Driver creation',
 'Driver creation',
 'Driver creation',
 'Driver creation',
 'Driver creation',
 'Dr

In [189]:
# Examine a timestamp field
records_list.date

['2016-02-18 19:39:50',
 '2016-02-16 14:04:00',
 '2016-02-15 15:39:46',
 '2016-02-15 14:56:52',
 '2016-02-08 15:06:52',
 '2015-12-30 14:01:57',
 '2015-11-19 09:59:18',
 '2015-11-06 16:16:51',
 '2015-11-06 15:31:58',
 '2015-10-30 21:48:14',
 '2016-02-23 14:47:44',
 '2016-02-23 14:39:06',
 '2016-02-23 14:28:15',
 '2016-02-23 14:10:46',
 '2016-02-23 13:52:11',
 '2016-02-23 13:39:57',
 '2016-02-23 12:25:38',
 '2016-02-23 12:21:27',
 '2016-02-23 11:30:51',
 '2016-02-23 03:17:15',
 '2016-02-23 03:16:42',
 '2016-02-23 01:26:16',
 '2016-02-23 01:24:10',
 '2016-02-23 01:20:52',
 '2016-02-23 01:18:22',
 '2016-02-23 01:15:01',
 '2016-02-23 00:03:36',
 '2016-02-22 22:08:13',
 '2016-02-22 21:35:11',
 '2016-02-22 20:53:30',
 '2016-02-22 20:50:54',
 '2016-02-22 20:12:02',
 '2016-02-22 18:52:37',
 '2016-02-22 18:29:58',
 '2016-02-22 15:00:14',
 '2016-02-22 14:54:52',
 '2016-02-22 14:54:41',
 '2016-02-22 14:32:39',
 '2016-02-22 14:10:44',
 '2016-02-22 13:53:27',
 '2016-02-22 13:38:47',
 '2016-02-22 13:

In [192]:
# mmmmm: dates are strings
type(objects.date[0])

str

In [191]:
# Not quite sure why this is useful, the ID are already there in the record
claims_model._get_external_ids()

{'__export__.crm_claim_375': <Record 'crm.claim,375'>,
 '__export__.crm_claim_449': <Record 'crm.claim,449'>,
 '__export__.crm_claim_1193': <Record 'crm.claim,1193'>,
 '__export__.crm_claim_935': <Record 'crm.claim,935'>,
 '__export__.crm_claim_403': <Record 'crm.claim,403'>,
 '__export__.crm_claim_652': <Record 'crm.claim,652'>,
 '__export__.crm_claim_811': <Record 'crm.claim,811'>,
 '__export__.crm_claim_169': <Record 'crm.claim,169'>,
 '__export__.crm_claim_1127': <Record 'crm.claim,1127'>,
 '__export__.crm_claim_274': <Record 'crm.claim,274'>,
 '__export__.crm_claim_197': <Record 'crm.claim,197'>,
 '__export__.crm_claim_323': <Record 'crm.claim,323'>,
 '__export__.crm_claim_609': <Record 'crm.claim,609'>,
 '__export__.crm_claim_40': <Record 'crm.claim,40'>,
 '__export__.crm_claim_891': <Record 'crm.claim,891'>,
 '__export__.crm_claim_193': <Record 'crm.claim,193'>,
 '__export__.crm_claim_146': <Record 'crm.claim,146'>,
 '__export__.crm_claim_401': <Record 'crm.claim,401'>,
 '__expo

## Conclusion

The only advantage I see using the plain XMLRPC library is that it returns an *enhanced* version of the records (one where the keys to other tables are enhanced with a descriptive field name). Other than that the ERPpeek wrapper is more powerful. 

The bottom line is: all we usually do is load an entire table inside a Pandas DataFrame object. So... mmm... Right now that's what the OdooConnector object does:

In [193]:
from connectors.odoo import OdooConnector
from pandas import ExcelWriter

odoo = OdooConnector()
claims_df = odoo.extract('crm.claim')
claims_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1359 entries, 0 to 1358
Data columns (total 40 columns):
__last_update           1359 non-null object
action_next             1359 non-null object
active                  1359 non-null bool
categ_id                1359 non-null object
cause                   1359 non-null object
company_id              1359 non-null object
create_date             1359 non-null object
create_uid              1359 non-null object
date                    1359 non-null object
date_action_next        1359 non-null object
date_closed             1359 non-null object
date_deadline           1359 non-null object
description             1359 non-null object
display_name            1359 non-null object
email_cc                1359 non-null bool
email_from              1359 non-null object
id                      1359 non-null int64
message_follower_ids    1359 non-null object
message_ids             1359 non-null object
message_is_follower     1359 non-null bool
